### Proving Theorems In Propositional Logic with LSTM-Based Text Generators
#### Author: Omar Afifi

Consider a number of propositional (i.e. variable-free) sentences (premsises). For example: 

1. p→q 
2. o
3. pv¬o

A propositional proof from the premises to a conclusion (another sentence) is a sequence of variable-free statements that follow from the premises by logical deduction rules (e.g. modus ponens, modus tollens, modus tollendo ponens, dysjuntive elimination, etc ... )

For example, a proof of the propositional sentence (q) from the preceding premises is as follows: 

4. ¬¬o (from 2, double negation)
5. p (from 3 and 4, dysjuntive elimination )
5. q (from 1 and 5, modus ponens)

QED.


This notebook explores the utility of using LSTM text-generators to generate a propositional proof given a collection of propositional sentences. Our hope is that it can be helpful as a stepping stone to making progress in the arena of stochastic theorem provers. 

Credits: Hugging Face User ergotts for building this dataset: https://huggingface.co/datasets/ergotts/propositional-logic


### Loading Data and preparring the input

In [130]:
import process_data

#load the data from hugging face mode = 'w' means that we are tokenizing words rather than characters or sentences. 
proofs_dataset = process_data.LoadLogicData(mode = 'w') 

#format the proofs: essentially just mapping words to integers and then creating n-gram sequences
word_to_int, int_to_word, sequenced_proofs = process_data.generate_sequences(proofs_dataset)

#split data into input and label by setting label equal to next word.
#sequence length is the length of eqch sequence, this allows us to pack them during training. 
X, sequence_lengths,y = process_data.makeXy(sequenced_proofs)


### making the data compatible with torch API

In [131]:
import torch as t
import torch.utils.data  as data

X = t.tensor(X, dtype = t.int64)
sequence_lengths =  t.tensor(sequence_lengths, dtype = t.int64)
y = t.tensor(y, dtype = t.int64)

torch_data = data.DataLoader(data.TensorDataset(X,sequence_lengths, y),
                            batch_size = 100)

### Loading the Model and Training

In [ ]:
import torch 
from torch import nn
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class LSTM(nn.Module):
    #constructor that inherits from nn.Module
    def __init__(self, seq_length,  hidden_size, num_layers,vocab_size):

        super(LSTM, self).__init__()
        #should probably initilize the hidden states
        self.seq_length = seq_length
        self.hidden_dim = hidden_size
        self.vocab_size = vocab_size
        self.num_layers = num_layers

        #we need to embed the words, a rule of thumb is that the 
        # embedding has the fourth root of the size of the vocabulary
        self.embedding = nn.Embedding(vocab_size, hidden_size)

        #initilize an lstm layer
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first = True)
        
        #output hidden layer
        self.fc = nn.Linear(hidden_size, self.vocab_size)

    def forward(self, X, sequence_lengths):
        """ forward pass through network"""

        X = self.embedding(X)

        X = pack_padded_sequence(X, sequence_lengths, 
                                 batch_first = True, 
                                 enforce_sorted = False)


        X, (H,C) = self.lstm(X)
        X, _ = pad_packed_sequence(X, batch_first = True)

        fc_out = self.fc(X)

        return F.log_softmax(fc_out, dim = -1)


    def train(self, train_loader, epochs, 
              loss_function, optimizer):

        for epoch in range(epochs): # for each epoch

            epoch_loss = 0
            correct_count = 0
            prediction_count = 0

            for index, data in enumerate(train_loader): #one pass over the training data

                X, sequence_lengths, y = data

                optimizer.zero_grad() # zero gradients to avoid blowup
                output = self.forward(X, sequence_lengths)

                output = output[range(len(X)), sequence_lengths-1]

                output = output.view(-1, self.vocab_size)

                #print(output.shape)

                loss = loss_function(output, y)

                loss.backward()
                #gradient clipping helps avoid blowup, which was a problem with training
                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=4)  
                optimizer.step()
   
                #update metrics
                epoch_loss += loss.item()
                _, y_hat = torch.max(output, dim = 1)
                correct_count += (y_hat == y).sum().item()
                prediction_count += y.size(0)



            #print metrics
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
            print(f'Epoch [{epoch+1}/{epochs}], Accuracy: {correct_count/prediction_count:.4f}')
            print('   ')

    
    def generateToken(self, x,l):
        "predict the next token from a sequence"

        with torch.no_grad():
            #this will be the last item in the prediciton vector
            predicted_sequence = self.forward(x,l)
            predicted_sequence = predicted_sequence[:,-1,:]
            return torch.argmax(predicted_sequence, dim = 1)[0] #return argmax{log(p_i) w.r.t i}




In [133]:

vocab_size = len(word_to_int)
hidden_size = 2
num_layers = 1
epochs = 1
loss_function = torch.nn.CrossEntropyLoss(reduction = "mean")

In [134]:

lstm = LSTM(seq_length = len(X[0]), 
            hidden_size = hidden_size, 
            num_layers = num_layers, 
            vocab_size = vocab_size)


optimizer = torch.optim.Adam(params = lstm.parameters(), lr = .01)
#lstm.train(torch_data, epochs, loss_function, optimizer)

Building a Proofs DataSet

Load the testing data from hugging face

In [135]:
def load_test_data():
    
    import pandas as pd
    test_data = pd.read_csv("hf://datasets/afifio/PropositionalProofs/proofs.csv").to_numpy()[:,0]
    test_data = [s.replace('\n', ' ').lower() for s in test_data]
    test_data = [s.replace(':', ' ').lower() for s in test_data]
    test_data = [s.replace('¬', '¬ ').lower() for s in test_data]
    test_data = [s.replace('(', '( ').lower() for s in test_data]
    test_data = [s.replace(')', ' )').lower() for s in test_data]
    for i in range(0,10):
        test_data = [s.replace(f'{i}.', '') for s in test_data]

    test_data = [s.split() for s in test_data]
    
    return test_data


def process_test_data(test_proofs, map):

    f = lambda word : map[word]

    test_proofs = [ [f(token) for token in proof ] for proof in test_proofs ]
    return [len(proof) for proof in test_proofs], test_proofs



In [136]:
test_data = load_test_data()
test_sequence_lengths, test_data = process_test_data(test_data, word_to_int)
test_data = process_data.pad(test_data, 0, X.shape[1])

In [137]:
sample, length = test_data[0], test_sequence_lengths[0]
length
sample = t.tensor([sample], dtype = t.int64)
length = t.tensor([length], dtype = t.int64)
length

tensor([8])

In [138]:
lstm.generateToken(sample, length)

tensor(49)